<a href="https://colab.research.google.com/github/NayanaMarques/ANN_2021/blob/master/Projeto_Nayana_RN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
#Projeto 1: Redes Neurais - Professor Ubirajara Filho
#Nayana G. Marques Silva
#Artigo: Comparison of neural network application for fuzzy and ANFIS approaches for multi-criteria decision making problems (2014)
#
#Definição de bibliotecas
#
import numpy as np #biblioteca numérica Phyton
import pandas as pd #biblioteca para manipulação de arquivos
import matplotlib.pyplot as plt #biblioteca para gráficos no Phyton
from sklearn.neural_network import MLPRegressor #biblioteca para classificação e regressão (regressão - MLPRegressor)
from sklearn.metrics import mean_squared_error #biblioteca para soma dos quadrados dos desvios
from sklearn.metrics import r2_score #biblioteca para avaliação do ajuste a partir do coeficiente de determinação
from google.colab import files #biblioteca para carregar arquivos no Phyton

In [48]:
!git clone https://github.com/NayanaMarques/ANN_2021
%cd ANN_2021

Cloning into 'ANN_2021'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 43 (delta 19), reused 9 (delta 3), pack-reused 0
Unpacking objects: 100% (43/43), done.
/content/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021/ANN_2021


In [49]:
# leitura dos dados
df=pd.read_csv("Dados de entrada.csv",delimiter=';')
print(df.head(34))

    Qual  Deliv  Tech  Price  Loc  Supplier Score Train or Test
0    3.0    6.0   3.5    4.0  1.5            0.05      Training
1    3.5    3.5   6.0    6.0  1.5            0.05          Test
2    3.0    4.0   5.0    4.0  7.0            0.07      Training
3    7.0    6.0   4.5    6.0  3.0            0.10          Test
4    6.5    6.5   6.5    6.5  5.5            0.11      Training
5    5.0    4.0   5.5    4.0  5.0            0.11          Test
6    5.0    6.0   6.0    5.5  2.5            0.12      Training
7    3.5    5.0   6.0    6.5  3.5            0.13          Test
8    5.5    4.5   6.0    4.5  7.0            0.15      Training
9    5.0    6.5   3.5    3.5  8.0            0.16          Test
10   5.5    6.5   5.5    7.0  5.5            0.19      Training
11   5.0    7.0   5.5    7.0  5.5            0.19          Test
12   4.0    6.0   7.0    5.0  9.0            0.19      Training
13   2.0    6.5   2.0    8.0  4.0            0.22          Test
14   5.5    7.5   5.5    3.5  7.5       

In [50]:
# funcao_normatiza_dados - normalizar dados (mesma ordem de grandeza)
def funcao_normatiza_dados(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max() #identifica valor máximo
        min_x=x[i].min() #identifica valor mínimo
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [51]:
# funcao_dados_originais - retorna os dados a forma original
def funcao_dados_originais(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [52]:
# separa os dados em normatizados e originais
Dat_Select=[]
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=funcao_normatiza_dados(Dat_Select)

In [53]:
DAT.head(3) 

AttributeError: ignored